Execution for house price estimation

In [1]:
# Import relevant files and libraries
import DataProcessor
import sys
import kmeans
import LRegression
import NLRegression
import performance_metrics

Define the parameters

In [19]:
# Allocate 80% of data to training
train_size = 0.8

# Identify features to be used
#  columnsToKeep = ['Price','Type','Bedroom','Bathroom','Car','Landsize','BuildingArea','Latitude','Longitude']
columnsToKeep = ['Price', 'Size']

# Path to dataset.
filepath = '../dataset/Melbourne_housing_FULL.csv'

# Identify prediction (ground truth)
prediction_column = 'Price'

Process raw data

In [20]:
# Create a DataProcessor object and load the data
dp_obj = DataProcessor.DataProcessor()
dp_obj.LoadData(filepath)

# Remove incomplete points, remove unused features and shuffle the data
dp_obj.keepSelectedColumns(columnsToKeep)
dp_obj.filterData()
dp_obj.shuffleData()

# Split the data into training and testing
train_X, train_Y, test_X, test_Y = dp_obj.splitData(train_size, prediction_column)

Data Processor object created
training size 7506
test size 1876


K-Means Clustering

In [ ]:
# Ashton

Linear Regression

In [ ]:
# Train
lr = LRegression.LRegression()
y_pred = lr.LinReg(features=train_X,target=train_Y,assignment=assignment)

In [ ]:
# Test


Non-Linear Regression (Gaussian Processing)

In [ ]:
# Train

In [ ]:
# Test

Evaluation Metrics

In [ ]:
# Wajeeha

User input for preference

In [12]:
# Dwelling type
type = input("What type of dwelling would you prefer? Either 'u' for unit or 'h' for house")
# Number of bedrooms
bedrooms = input("How many bedrooms would you prefer?")
# Number of bathrooms
bathrooms = input("How many bathrooms would you prefer?")
# Number of car parking spaces
car = input("How many car parking spaces would you prefer?")
# Size of land
size = input("What size would you prefer?")
# Latitude
latitude = input("Would you prefer 'east' or 'west'?")
# Longitude
longitude = input("Would you prefer 'north' or 'south'?")

Final prediction

In [ ]:
# Return predicted value from best performing model
targets = [type,bedrooms,bathrooms,car,size,latitude,longitude]
pred = lr.predict(features=targets)